In [19]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim

In [20]:
import glob
path = "./../dist/char*.txt"
contents=[]
for filename in glob.glob(path):
    with open(filename, 'r') as f:
        for line in f:
            contents.append(line.strip('\n'))
print(len(contents))
i=0
while (i<len(contents)-1):
    contents[i]=float(contents[i])
    i+=1
    if ((i+1)%15==0):
        i=i+1

795


In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [22]:
from random import randint as r

In [23]:
i=0
for a in range(0, 49):
    i=r(0, (len(contents)/15-1))
    tt=torch.tensor(contents[(a*15):(a*15+14)])
    print(a, ' : ', tt)

0  :  tensor([1.7000e+02, 1.9600e+02, 2.3915e+02, 2.7594e+02, 4.2629e+00, 4.6510e+00,
        1.8965e+01, 1.6539e+01, 9.7704e+04, 7.7159e+03, 9.8711e+00, 7.7954e-01,
        5.7453e-02, 4.9622e-02])
1  :  tensor([1.3700e+02, 2.0300e+02, 1.9245e+02, 2.8584e+02, 4.7254e+00, 4.8075e+00,
        1.6646e+01, 1.8780e+01, 9.7918e+04, 5.9891e+03, 9.8917e+00, 6.0503e-01,
        6.9876e-02, 4.8330e-02])
2  :  tensor([1.2100e+02, 1.8000e+02, 1.6981e+02, 2.5330e+02, 4.9331e+00, 4.3437e+00,
        1.5881e+01, 1.8789e+01, 9.7496e+04, 7.2560e+03, 9.8272e+00, 7.3138e-01,
        7.9711e-02, 5.4328e-02])
3  :  tensor([1.6400e+02, 1.7700e+02, 2.3065e+02, 2.4905e+02, 4.4259e+00, 4.2629e+00,
        1.8905e+01, 1.8064e+01, 9.8584e+04, 9.4879e+03, 9.8960e+00, 9.5251e-01,
        5.9634e-02, 5.5542e-02])
4  :  tensor([2.0500e+02, 2.1200e+02, 1.3217e+02, 1.3670e+02, 4.9991e+00, 4.9991e+00,
        1.7822e+01, 1.7248e+01, 9.8096e+04, 7.5926e+03, 9.8987e+00, 7.6623e-01,
        4.7512e-02, 4.5995e-02])
5  : 

In [24]:
import time

In [25]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(in_features=14, out_features=14, bias=True)
        self.fc2 = nn.Linear(in_features=14, out_features=14, bias=True)
        self.out = nn.Linear(in_features=14, out_features = 4, bias=True)
        
    def forward(self, t):
        #1st layer
        t = self.fc1(t)
        #2nd layer
        t = self.fc2(t)
        #output layer
        t = self.out(t)
        
        return t

In [26]:
net=Network()
params=list(net.parameters())
criterion = nn.MSELoss()
net.zero_grad()
optimizer = optim.SGD(net.parameters(), lr=0.0000000005)
lossDuplicationCounter = 0
prevloss=0
loss = 4
i=0

In [27]:
start=time.time()
while ((loss > 0.01) and (lossDuplicationCounter<10)):
    a=r(0, (len(contents)/15-1))
    chars = torch.tensor(contents[(a*15):(a*15+14)])
    optimizer.zero_grad()   # zero the gradient buffers
    out = net(chars)
    if (contents[a*14+15]=='Marcin'):
        target = torch.tensor([1., 0., 0., 0.])
    elif (contents[a*14+15]=='Bartek'):
        target = torch.tensor([0., 1., 0., 0.])
    elif (contents[a*14+15]=='Maciek'):
        target = torch.tensor([0., 0., 1., 0.,])
    else:
        target = torch.tensor([0., 0., 0., 1.,])
    loss = criterion(out, target)
    if prevloss == loss:
        lossDuplicationCounter +=1
        print('loss duplicate #', lossDuplicationCounter)
    else:
        lossDuplicationCounter = 0
    prevloss = loss
    loss.backward(retain_graph=True)
    #loss.backward()
    optimizer.step()    # Does the update
    if i%1000==0:
        print("\n\niteration: ", i, "\nloss: ", loss, '\n\tOutput:\n', out)
    i+=1
print("Learning time: \n\t", time.time()-start)
torch.cuda.empty_cache()



iteration:  0 
loss:  tensor(18304984., grad_fn=<MseLossBackward>) 
	Output:
 tensor([ -621.2948, -3564.9189,  2721.1582, -7259.8940],
       grad_fn=<AddBackward0>)


iteration:  1000 
loss:  tensor(802.6117, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-11.2109,   6.9219,  54.7534,  -5.2382], grad_fn=<AddBackward0>)


iteration:  2000 
loss:  tensor(1591.9047, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-23.9584,   5.3503,  73.9562, -16.1891], grad_fn=<AddBackward0>)


iteration:  3000 
loss:  tensor(4193.7837, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-57.5021,  -5.2001, 107.1790, -43.2071], grad_fn=<AddBackward0>)


iteration:  4000 
loss:  tensor(651.2397, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-16.9041,   2.0704,  47.0386, -10.2681], grad_fn=<AddBackward0>)


iteration:  5000 
loss:  tensor(829.8742, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-18.9517,  -2.9757,  53.1846, -10.0849], grad_fn=<AddBackward0>)


iteration:  6000 
loss:  tensor(115.4474, grad_fn=<Ms



iteration:  54000 
loss:  tensor(60.3276, grad_fn=<MseLossBackward>) 
	Output:
 tensor([8.4327, 5.4482, 8.9799, 8.7382], grad_fn=<AddBackward0>)


iteration:  55000 
loss:  tensor(64.0411, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 6.0033, 10.0326,  9.8986,  6.3682], grad_fn=<AddBackward0>)


iteration:  56000 
loss:  tensor(15.0747, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-0.1465,  7.0599,  3.2136,  0.6708], grad_fn=<AddBackward0>)


iteration:  57000 
loss:  tensor(22.4483, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-1.2849,  7.6876,  1.2255, -4.2479], grad_fn=<AddBackward0>)


iteration:  58000 
loss:  tensor(116.4520, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 7.4523,  1.7234, 13.6175, 15.8952], grad_fn=<AddBackward0>)


iteration:  59000 
loss:  tensor(38.3247, grad_fn=<MseLossBackward>) 
	Output:
 tensor([7.3332, 4.2174, 6.7521, 7.0121], grad_fn=<AddBackward0>)


iteration:  60000 
loss:  tensor(69.6572, grad_fn=<MseLossBackward>) 
	Output:
 tensor([8.1752, 6.7



iteration:  109000 
loss:  tensor(17.8937, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-4.6372, -5.8800, -3.3185, -1.1176], grad_fn=<AddBackward0>)


iteration:  110000 
loss:  tensor(15.4336, grad_fn=<MseLossBackward>) 
	Output:
 tensor([5.6803, 4.1721, 0.3570, 4.4546], grad_fn=<AddBackward0>)


iteration:  111000 
loss:  tensor(17.6613, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-4.3449, -3.7506, -4.5967, -3.0708], grad_fn=<AddBackward0>)


iteration:  112000 
loss:  tensor(44.7386, grad_fn=<MseLossBackward>) 
	Output:
 tensor([6.8396, 7.4430, 7.1237, 6.1018], grad_fn=<AddBackward0>)


iteration:  113000 
loss:  tensor(1.5117, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-0.3330, -2.2089,  0.9604,  1.3665], grad_fn=<AddBackward0>)


iteration:  114000 
loss:  tensor(44.0543, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-7.9434, -5.8509, -6.5808, -4.9649], grad_fn=<AddBackward0>)


iteration:  115000 
loss:  tensor(48.8112, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 2.126



iteration:  164000 
loss:  tensor(39.7196, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-0.3893,  3.6690,  7.9693, 10.0419], grad_fn=<AddBackward0>)


iteration:  165000 
loss:  tensor(33.4851, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 1.0705, -5.5700, -8.0833, -5.0357], grad_fn=<AddBackward0>)


iteration:  166000 
loss:  tensor(80.9957, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 7.7091,  5.5948, 10.9768, 11.6189], grad_fn=<AddBackward0>)


iteration:  167000 
loss:  tensor(42.0431, grad_fn=<MseLossBackward>) 
	Output:
 tensor([7.8097, 4.2844, 6.0728, 8.2073], grad_fn=<AddBackward0>)


iteration:  168000 
loss:  tensor(44.2799, grad_fn=<MseLossBackward>) 
	Output:
 tensor([6.5596, 6.7699, 7.1599, 7.0825], grad_fn=<AddBackward0>)


iteration:  169000 
loss:  tensor(13.8370, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 2.4407, -3.6679, -4.9624, -2.3634], grad_fn=<AddBackward0>)


iteration:  170000 
loss:  tensor(145.0559, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-13.



iteration:  219000 
loss:  tensor(43.5195, grad_fn=<MseLossBackward>) 
	Output:
 tensor([6.8822, 4.8894, 7.1791, 8.1601], grad_fn=<AddBackward0>)


iteration:  220000 
loss:  tensor(6.1594, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-0.7693, -0.2837, -4.7148, -0.3175], grad_fn=<AddBackward0>)


iteration:  221000 
loss:  tensor(144.5624, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ -9.3674,  -8.1560, -16.7136, -11.0265], grad_fn=<AddBackward0>)


iteration:  222000 
loss:  tensor(147.2565, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ -7.6579,  -8.4142, -17.8207, -10.9167], grad_fn=<AddBackward0>)


iteration:  223000 
loss:  tensor(12.9280, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-4.2481, -4.4588, -2.2636, -1.9430], grad_fn=<AddBackward0>)


iteration:  224000 
loss:  tensor(4.6591, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 3.1373,  1.5803, -2.4752,  1.4123], grad_fn=<AddBackward0>)


iteration:  225000 
loss:  tensor(31.6583, grad_fn=<MseLossBackward>) 
	Output:
 t



iteration:  274000 
loss:  tensor(12.4540, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-1.2735, -5.4481, -3.9243, -0.7641], grad_fn=<AddBackward0>)


iteration:  275000 
loss:  tensor(3.9381, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 0.9416,  2.7724,  2.4112, -0.1685], grad_fn=<AddBackward0>)


iteration:  276000 
loss:  tensor(3.9977, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-1.7598, -3.1087, -0.6960, -0.5942], grad_fn=<AddBackward0>)


iteration:  277000 
loss:  tensor(43.5908, grad_fn=<MseLossBackward>) 
	Output:
 tensor([5.2629, 2.1733, 8.1973, 9.6456], grad_fn=<AddBackward0>)


iteration:  278000 
loss:  tensor(24.0806, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-0.2656,  8.4828,  4.7752, -0.2215], grad_fn=<AddBackward0>)


iteration:  279000 
loss:  tensor(2.8452, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-0.4248,  2.1981,  2.2161,  2.2073], grad_fn=<AddBackward0>)


iteration:  280000 
loss:  tensor(4.1161, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 2.81



iteration:  329000 
loss:  tensor(10.2539, grad_fn=<MseLossBackward>) 
	Output:
 tensor([3.6727, 2.5665, 2.6794, 4.7095], grad_fn=<AddBackward0>)


iteration:  330000 
loss:  tensor(5.5829, grad_fn=<MseLossBackward>) 
	Output:
 tensor([-2.8123,  1.1282,  3.6262,  1.0179], grad_fn=<AddBackward0>)


iteration:  331000 
loss:  tensor(2.5508, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ 0.7402,  3.0213, -0.0503,  0.2759], grad_fn=<AddBackward0>)


iteration:  332000 
loss:  tensor(20.9824, grad_fn=<MseLossBackward>) 
	Output:
 tensor([5.4941, 4.8145, 3.5743, 5.2177], grad_fn=<AddBackward0>)


iteration:  333000 
loss:  tensor(50.0778, grad_fn=<MseLossBackward>) 
	Output:
 tensor([  0.4278,  -5.9514, -11.3489,  -4.9927], grad_fn=<AddBackward0>)


iteration:  334000 
loss:  tensor(45.8650, grad_fn=<MseLossBackward>) 
	Output:
 tensor([7.1064, 8.5658, 5.8996, 5.9782], grad_fn=<AddBackward0>)


iteration:  335000 
loss:  tensor(116.9135, grad_fn=<MseLossBackward>) 
	Output:
 tensor([ -2.88

In [28]:
print(out)
print('\n', i, '\n', out, '\n', loss)

tensor([ 0.0542, -0.0643, -0.0803,  0.9226], grad_fn=<AddBackward0>)
340407 tensor(0.0049, grad_fn=<MseLossBackward>)


In [29]:
torch.save(net, "net.dat")
torch.save(net.state_dict(), "netstate.dat")

D:\Anaconda3\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Network. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [30]:
haba = torch.load("net.dat")
baba = Network()
baba.load_state_dict(torch.load("netstate.dat"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])